In [1]:
#dependencies
#for consistency with the course I am keeping all api keys in one file named config.py
import requests
from config import census_api_key
import pandas as pd

### Census Api basic info
Here is the link to the American Community Survey API for 2011-2017
    - https://www.census.gov/data/developers/data-sets/acs-1year.html

Chart with all the types of variables you can look up
    - https://api.census.gov/data/2017/acs/acs1.html
Variables - breaking down the data by sex/age/ethnicity
    - https://api.census.gov/data/2017/acs1/variables.html
Geographies - breaking down the data by different geographic areas/sizes
    - https://api.census.gov/data/2017/acs/acs1/geography.html


In [2]:
#Census Data Notes
#base API structure
    #base url + year + data set + data type (variable) + geographic area + API key
    #if you want to show all of a variable type <*> instead of a number/code
        #example - to get information for all states you would type <in=state:*>
base_url = "https://api.census.gov/data"
years = [2012,2013,2014,2015]
data_set = "acs5" #American Community Survey
#state codes for census
#https://www.census.gov/govs/www/gidstate.html
    #Illinois = 17

#Variable types (aka age/sex/ethnicity) for 1-year American Community Surveys
#api.census.gov/data/2013/acs1/variables.html

data_type = "B01003_001E" #total population
#data_type_error = "B01003_001M" #population margin of error

population_df=pd.DataFrame()

Notes: 
The Chicago crime database has data from 2001-present, American Community Surveys contain data up to 2017.
We originally were looking at data from 2011-2017, but data from 2011 and earlier has different community areas than 2012 on. Additionally the data is not available yet when we try to search for 2016 and 2017, so we are looking at data from 2012-2015.

In [3]:
#Chicago has 77 Community Areas
#state public use micro area has combined community areas
#example from census website
#https://api.census.gov/data/2013/acs1?get=NAME,B01001_001E&for=public%20use%20microdata%20area:*&in=state:*&key=YOUR_KEY_GOES_HERE

geographic_area = "public%20use%20microdata%20area"

In [4]:
#loop through years
for year in years:
    try:
        #build query url
        query_url = f"{base_url}/{year}/{data_set}?get=NAME,{data_type}&for={geographic_area}:*&in=state:17&key={census_api_key}"
        #make request
        res = requests.get(query_url)
        res = res.json()
        #change response into dataframe
        df = pd.DataFrame.from_records(res)
        #add year column to datafram
        df["year"] = year
        #add to population_df
        population_df = population_df.append(df, sort="True")
    except ValueError:
        print(f"Error {year}")

In [5]:
#From try/except errors and df
    #Errors with years 2016,2017 - data not avaiable message from census API
    #different data sorting 2011
#Only using years 2012-2015

In [6]:
population_df.head()

,0,1,2,3,year
0,NAME,B01003_001E,state,public use microdata area,2012
1,"Jo Daviess, Carroll, Whiteside & Lee Counties ...",132179,17,00104,2012
2,"Rock Island County PUMA, Illinois",147504,17,00105,2012
3,"Henry, Mercer, Henderson, Warren, Hancock, McD...",180417,17,00202,2012
4,"Adams, Pike, Brown, Schuyler & Mason Counties ...",112582,17,00300,2012


In [7]:
#Use value_counts to make sure all the public use microdata areas are represented each year
    #this is how we saw that the 2011 data was different from 2012-2015
population_df[3].value_counts()

03503    4
00401    4
03522    4
02300    4
03409    4
01104    4
02400    4
03523    4
03602    4
03532    4
01602    4
03527    4
03502    4
03529    4
03411    4
02200    4
03208    4
03526    4
03410    4
03203    4
03007    4
03209    4
02901    4
03420    4
03005    4
02801    4
00105    4
03422    4
03306    4
02000    4
        ..
01500    4
03528    4
03408    4
03416    4
03418    4
03407    4
03501    4
03525    4
03204    4
01900    4
00202    4
01001    4
00900    4
03107    4
03106    4
03521    4
02601    4
03307    4
03520    4
03308    4
03108    4
01205    4
03309    4
03700    4
03009    4
03202    4
02100    4
03531    4
03524    4
03102    4
Name: 3, Length: 89, dtype: int64

In [8]:
#rename columns
population_df = population_df.rename(columns={1:"Total Population",2:"State",3:"Public Use Microdata Area", "year":"Year"})
population_df.head()

,0,Total Population,State,Public Use Microdata Area,Year
0,NAME,B01003_001E,state,public use microdata area,2012
1,"Jo Daviess, Carroll, Whiteside & Lee Counties ...",132179,17,00104,2012
2,"Rock Island County PUMA, Illinois",147504,17,00105,2012
3,"Henry, Mercer, Henderson, Warren, Hancock, McD...",180417,17,00202,2012
4,"Adams, Pike, Brown, Schuyler & Mason Counties ...",112582,17,00300,2012


In [9]:
#Set index to public use microdata/name column
population_df = population_df.set_index([0])


In [10]:
#New dataframe
population_df.head()

,Total Population,State,Public Use Microdata Area,Year
0,,,,
NAME,B01003_001E,state,public use microdata area,2012
"Jo Daviess, Carroll, Whiteside & Lee Counties PUMA; Illinois",132179,17,00104,2012
"Rock Island County PUMA, Illinois",147504,17,00105,2012
"Henry, Mercer, Henderson, Warren, Hancock, McDonough & Fulton Counties PUMA; Illinois",180417,17,00202,2012
"Adams, Pike, Brown, Schuyler & Mason Counties PUMA; Illinois",112582,17,00300,2012


In [11]:
#Remove rows with census header names
illinois_population_df = population_df.drop(["NAME"])
#check to see if that worked
illinois_population_df.head()
illinois_population_df["Public Use Microdata Area"].value_counts()

03503    4
03008    4
03209    4
03522    4
02300    4
03409    4
01104    4
03501    4
03523    4
03602    4
03532    4
01602    4
03527    4
03502    4
03529    4
03411    4
02200    4
03208    4
03526    4
03410    4
03203    4
02901    4
03420    4
03421    4
03005    4
02801    4
00105    4
03422    4
03306    4
02000    4
        ..
03601    4
01500    4
03528    4
03408    4
03416    4
03202    4
03407    4
03525    4
01900    4
03412    4
03520    4
00600    4
01001    4
00900    4
03107    4
03106    4
03521    4
02601    4
03307    4
00202    4
03108    4
03531    4
01205    4
03309    4
03700    4
03009    4
03308    4
02100    4
03524    4
03102    4
Name: Public Use Microdata Area, Length: 88, dtype: int64

In [12]:
#reset index
illinois_population_df = illinois_population_df.reset_index()
illinois_population_df.head()

,0,Total Population,State,Public Use Microdata Area,Year
0,"Jo Daviess, Carroll, Whiteside & Lee Counties ...",132179,17,00104,2012
1,"Rock Island County PUMA, Illinois",147504,17,00105,2012
2,"Henry, Mercer, Henderson, Warren, Hancock, McD...",180417,17,00202,2012
3,"Adams, Pike, Brown, Schuyler & Mason Counties ...",112582,17,00300,2012
4,"Cass, Morgan, Scott, Greene, Macoupin, Jersey ...",143980,17,00401,2012


In [14]:
#rename Name column
illinois_population_df = illinois_population_df.rename(columns={0:"Name"})
illinois_population_df.head()

,Name,Total Population,State,Public Use Microdata Area,Year
0,"Jo Daviess, Carroll, Whiteside & Lee Counties ...",132179,17,00104,2012
1,"Rock Island County PUMA, Illinois",147504,17,00105,2012
2,"Henry, Mercer, Henderson, Warren, Hancock, McD...",180417,17,00202,2012
3,"Adams, Pike, Brown, Schuyler & Mason Counties ...",112582,17,00300,2012
4,"Cass, Morgan, Scott, Greene, Macoupin, Jersey ...",143980,17,00401,2012


In [16]:
#drop "State" column
illinois_population_df = illinois_population_df.drop(["State"], axis=1)
illinois_population_df.head()

,Name,Total Population,Public Use Microdata Area,Year
0,"Jo Daviess, Carroll, Whiteside & Lee Counties ...",132179,00104,2012
1,"Rock Island County PUMA, Illinois",147504,00105,2012
2,"Henry, Mercer, Henderson, Warren, Hancock, McD...",180417,00202,2012
3,"Adams, Pike, Brown, Schuyler & Mason Counties ...",112582,00300,2012
4,"Cass, Morgan, Scott, Greene, Macoupin, Jersey ...",143980,00401,2012


In [25]:
#pivot table
illinois_population_pivot = illinois_population_df.pivot(index ="Name", columns="Year", values="Total Population")
illinois_population_pivot.head()


Year,2012,2013,2014,2015
Name,,,,
"Adams, Pike, Brown, Schuyler & Mason Counties PUMA; Illinois",112582,112391,111998,111521
"Cass, Morgan, Scott, Greene, Macoupin, Jersey & Calhoun Counties PUMA; Illinois",143980,143394,142662,141551
"Champaign County--Champaign & Urbana Cities PUMA, Illinois",200931,202428,204214,205766
"Chicago City (Central)--Near North Side, Loop & Near South Side PUMA; Illinois",127124,132871,136968,141554
"Chicago City (North)--Edgewater, Uptown & Rogers Park PUMA; Illinois",167887,167162,164391,164412


In [24]:
#export data to csv
illinois_population_pivot.to_csv("Illinois_Population.csv")